In [13]:
######################### IMPORTANTE #################################

# SE NECESITA CONFIGURAR COMO AREA DE TRABAJO LA CARPETA QUE CONTIENE EL EXCEL VACIO Y EL CÓDIGO 
# (AMBOS EN LA MISMA CARPETA)

import os
os.chdir("C:/Users/flavi/OneDrive/Desktop/estasi")

In [ ]:
from bs4 import BeautifulSoup # importa la libreria Beautifulsoup
import requests               # importa libreria requests
import pandas as pd           # importa la libreria pandas
import openpyxl               # importa la libreria openpyxl

archivo_excel_vacio = pd.read_excel("TablaVacia.xlsx",header=1,index_col=0) # leer excel de cabecera fila 1 y indice columna 0
lista_genes_abreviados=list(archivo_excel_vacio["GEN ABREV"]) # importamos la columna que tiene los genes abreviados y los introducimos a una lista
m=3  # se incia en la fila 3
for i in lista_genes_abreviados: # lee cada abreviación del gen
    URL=f"https://www.uniprot.org/uniprot/?query={i.upper()}+bos+taurus&sort=score" # introducimos el url de busqueda de la pagina requerida
    page=requests.get(URL) # conección a la pagina
    soup=BeautifulSoup(page.content,"html.parser") # almacena el codigo html

    link_entry=soup.find_all("td",class_="entryID") # lista de entradas que aparacen en el resultado de la busqueda
    indice=[]  # genera una lista vacia
    for P in link_entry: # para cada entrada que aparece en el resultado de busqueda se almacena en la lista indice
        indice.append(P.text) # añade los resultados a la lista vacia
    try:  # si hay un resultado en el indice va intentar ejecutar el siguiente codigo
        FINAL=indice[0]   # selecciona el primer resultado
        url=f"https://www.uniprot.org/uniprot/{FINAL}" # introduce el url del primer resultado
        page=requests.get(url)  # conección a la pagina
        soup=BeautifulSoup(page.content,"html.parser") # almacena el codigo html
               
        #ENTRY Y ENTRY NAME
        entry_name=soup.find_all("h2",class_="page-title") # busca todos los elementos que tengan etiqueta h2 
        for Q in entry_name: # limpia los resultados y extrae unicamente los textos
            entry1=Q.text # extrae unicamente los textos
        entry1=entry1.split() # extrae unicamente los textos
        ENTRY=entry1[2] # selecciona el elemento número 2 de los textos
        ENTRY_NAME="".join(list(entry1[3])[1:len(list(entry1[3]))-1]) # introduce lo encontrado anteriormente y lo selecciona como entry_name

        #PROTEINA
        protein=soup.find_all("h1",property="name") # busca todos los elementos que tenga etiqueta h1
        for Q in protein:  # limpia los resultados 
            PROTEIN=Q.text # extrae unicamente los textos

        #GEN
        gen_abrev=soup.find_all("div",class_='entry-overview-content',id="content-gene") # busca todos los elementos que contienen id "content-gene", con etiqueta div.
        for Q in gen_abrev: # limpia los resultados
            GEN_ABREV=Q.text # extrae unicamente los textos

        #ORGANISMO
        organismo=soup.find_all("div",class_='entry-overview-content',id="content-organism") # busca todos los elementos que contienen id "content-organism", con etiqueta div.
        for Q in organismo: # limpia los resultados
            ORGANISMO=Q.text # extrae unicamente los textos

        #STATUS
        status=soup.find_all("span",class_='context-help bin-score tooltipped-click') # busca todos los elementos, con etiqueta span.
        for Q in status: # limpia los resultados
            STATUS_PRE=Q.text # extrae unicamente los textos
        STATUS=STATUS_PRE.split("\n")[0].split(":")[1] # selecciona el status de toda esa respuesta

        #GO BIOLOGICAL PROCESS
        bio_process=soup.find_all("a",onclick="window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');") # busca todos los elementos, con etiqueta a
        BIO_PROCESS=[] # crea una lista
        for Q in bio_process: # limpia los resultados
            BIO_PROCESS.append(Q.text) # selecciona los textos y los añade a la lista vacia
        BIO_PROCESS=str(BIO_PROCESS) # vuelve toda la lista en una cadena 
         
        
        

    except:                 # si es que no hay resultado en la busqueda dejara en blanco las celdas de los siguientes atributos
        ENTRY=""
        ENTRY_NAME=""
        PROTEIN=""
        GEN_ABREV=""
        ORGANISMO=""
        STATUS=""
        BIO_PROCESS=""
        
        excel=openpyxl.load_workbook('TablaVacia.xlsx') # abre el excel para completar los datos obtenidos en las columnas
        sheet=excel['Hoja1']   # hoja 1 del excel
        sheet.cell(row=m,column=3).value = ENTRY    # llena los valores 
        sheet.cell(row=m,column=4).value = ENTRY_NAME
        sheet.cell(row=m,column=5).value = PROTEIN
        sheet.cell(row=m,column=6).value = GEN_ABREV
        sheet.cell(row=m,column=7).value = ORGANISMO
        sheet.cell(row=m,column=8).value = STATUS
        sheet.cell(row=m,column=9).value = BIO_PROCESS
        excel.save('TablaVacia.xlsx')   # guarda el excel con los datos obtenidos  
        m=m+1  # pasa a la siguiente fila